In [18]:
# plotly with inputs

import dash
import pandas as pd
import plotly.graph_objects as go
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [4]:
airline_df = pd.read_csv('./airline_data.csv')
airline_df.shape

(27000, 110)

In [5]:
airline_df.columns

Index(['Unnamed: 0', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'FlightDate', 'Reporting_Airline', 'DOT_ID_Reporting_Airline',
       'IATA_CODE_Reporting_Airline',
       ...
       'Div4WheelsOff', 'Div4TailNum', 'Div5Airport', 'Div5AirportID',
       'Div5AirportSeqID', 'Div5WheelsOn', 'Div5TotalGTime',
       'Div5LongestGTime', 'Div5WheelsOff', 'Div5TailNum'],
      dtype='object', length=110)

In [6]:
pd.set_option('display.max_columns', None)
airline_df.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1295781,1998,2,4,2,4,1998-04-02,AS,19930,AS,N785AS,584,11884,1188401,31884,GEG,"Spokane, WA",WA,53.0,Washington,93,14747,1474702,30559,SEA,"Seattle, WA",WA,53.0,Washington,93,1330,1330.0,0.0,0.0,0.0,0.0,1300-1359,8.0,1338.0,1415.0,5.0,1426,1420.0,-6.0,0.0,0.0,-1.0,1400-1459,0.0,NaN,0.0,56.0,50.0,37.0,1.0,224.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1125375,2013,2,5,13,1,2013-05-13,EV,20366,EV,N24103,4132,11618,1161802,31703,EWR,"Newark, NJ",NJ,34.0,New Jersey,21,14524,1452401,34524,RIC,"Richmond, VA",VA,51.0,Virginia,38,1301,1255.0,-6.0,0.0,0.0,-1.0,1300-1359,9.0,1304.0,1358.0,13.0,1423,1411.0,-12.0,0.0,0.0,-1.0,1400-1459,0.0,NaN,0.0,82.0,76.0,54.0,1.0,277.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,118824,1993,3,9,25,6,1993-09-25,UA,19977,UA,NaN,2206,14108,1410801,34108,PIA,"Peoria, IL",IL,17.0,Illinois,41,13930,1393001,30977,ORD,"Chicago, IL",IL,17.0,Illinois,41,1650,1723.0,33.0,33.0,1.0,2.0,1600-1659,NaN,NaN,NaN,NaN,1730,1815.0,45.0,45.0,1.0,3.0,1700-1759,0.0,NaN,0.0,40.0,52.0,NaN,1.0,130.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,634825,1994,4,11,12,6,1994-11-12,HP,19991,HP,NaN,1207,12892,1289201,32575,LAX,"Los Angeles, CA",CA,6.0,California,91,14107,1410701,30466,PHX,"Phoenix, AZ",AZ,4.0,Arizona,81,1245,1309.0,24.0,24.0,1.0,1.0,1200-1259,NaN,NaN,NaN,NaN,1457,1538.0,41.0,41.0,1.0,2.0,1400-1459,0.0,NaN,0.0,72.0,89.0,NaN,1.0,370.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1888125,2017,3,8,17,4,2017-08-17,UA,19977,UA,N827UA,576,11003,1100303,31003,CID,"Cedar Rapids/Iowa City, IA",IA,19.0,Iowa,61,11292,1129202,30325,DEN,"Denver, CO",CO,8.0,Colorado,82,755,746.0,-9.0,0.0,0.0,-1.0,0700-0759,8.0,754.0,836.0,8.0,902,844.0,-18.0,0.0,0.0,-2.0,0900-0959,0.0,NaN,0.0,127.0,118.0,102.0,1.0,692.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [24]:
# create a Dash application:
app = dash.Dash(__name__)

# create the layout of the application:
app.layout = html.Div(children=[
    html.H1('Airline Performance Dashboard', style={'textAlign': 'center', 'color': 'red', 'font-size': 40}),
    html.Div(["Input Year: ",
              dcc.Input(id="inputyear_id", value="2010", type="number", style={'height': '50px', 'width': '350px'}, )],
             style={'font-size': 30, 'color': 'red'}),
    html.Br(),
    html.Div(dcc.Graph(id='linegraph_id'))
])


# callback decorator: user-defined dn should be called whenever the input changes
@app.callback(
    Output(component_id='linegraph_id', component_property='figure'),
    Input(component_id='inputyear_id', component_property='value')
)
def create_graph(entered_year):
    try:
        year = int(entered_year)
    except ValueError:
        year = 2010  # Default value if input is invalid
    # sub-setting the data based on the entered year
    df = airline_df[airline_df['Year'] == year]

    if df.empty:
        # Handle the case where no data is available for the selected year
        fig = go.Figure()
        fig.update_layout(title='No data available for the selected year')
        return fig

    # grouping the data by month and computing the average arrival delay
    linegraph_df = df.groupby('Month').aggregate({'ArrDelay': 'mean'}).reset_index()

    fig = go.Figure(go.Scatter(x=linegraph_df['Month'], y=linegraph_df['ArrDelay'], mode='lines+markers',
                               marker=dict(color='green')))
    fig.update_layout(title='Month vs Average-Delay in minutes', xaxis_title='Month', yaxis_title='Arrival Delay')
    return fig


# Run the application
if __name__ == '__main__':
    print('Running the application...')
    app.run_server(port=8051)

Running the application...
